# Analyse von Zeitreihen mit saisonalen Mustern


### Zunächst: Dekomposition von Trend, Saisonalen Anteilen und Zufalls-(Rausch-)anteilen

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Erstellen einer Beispiel-Zeitreihe
np.random.seed(0)
n = 120
t = np.arange(n)
trend = 0.1 * t
seasonal_pattern = 10 * np.sin(2 * np.pi * t / 12)  # Monatlich mit einer 12-monatigen Periode
noise = np.random.normal(0, 1, n)
time_series = trend + seasonal_pattern + noise

# Konvertieren der Zeitreihe in ein Pandas DataFrame
data = pd.DataFrame({'TimeSeries': time_series}, index=pd.date_range(start='2000-01-01', periods=n, freq='M'))

# Additive Trenddekomposition
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data, model='additive')

# Visualisieren der Komponenten
plt.figure(figsize=(12, 6))
plt.subplot(411)
plt.plot(data['TimeSeries'], label='Originalzeitreihe')
plt.legend()
plt.subplot(412)
plt.plot(result.trend, label='Trend')
plt.legend()
plt.subplot(413)
plt.plot(result.seasonal, label='Saison')
plt.legend()
plt.subplot(414)
plt.plot(result.resid, label='Rest')
plt.legend()
plt.tight_layout()
plt.show()


# Prognosemodelle

### Beispiel zu Autoregression AR(1)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Beispielzeitreihendaten
time_series = np.array([10, 12, 15, 18, 20, 22, 25, 28, 30, 32])

# Verzögerte Werte für die Autoregression erstellen (zum Beispiel, verwenden wir die letzten 3 Werte)
X = []
y = []
for i in range(lags, len(time_series)):
    X.append(time_series[i-1:i])  #Vom Vorgänger....
    y.append(time_series[i])   #...soll auf den Nachfolger geschlossen werden können.

X = np.array(X)
y = np.array(y)

# Linear Regression Modell anpassen
model = LinearRegression()
model.fit(X, y)

# Vorhersagen für den nächsten Zeitschritt
next_step = np.array([time_series[-lags:]])
predicted_value = model.predict(next_step)

# Plotten der Zeitreihe und Vorhersagen
plt.plot(time_series, label='Zeitreihe')
plt.xlabel('Zeitschritte')
plt.ylabel('Wert')
plt.title('Zeitreihe und Vorhersagen')
plt.scatter(len(time_series) - 1, predicted_value, color='red', label='Vorhersage (Ende)')
plt.scatter(0, time_series[0], color='green', label='Anfang')
plt.legend()
plt.show()


In [ ]:
X

In [ ]:
y

### Einfache Autoregression mit einem ARIMA-Modell (AutoRegressIve, Moving Average)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Erstellen einer Zeitreihe mit saisonalem Muster (z.B. jährlich)
np.random.seed(0)
n = 100
t = np.arange(n)
seasonal_pattern = 10 * np.sin(2 * np.pi * t / 12) + t/10  # Monatlich mit einer 12-monatigen Periode
noise = np.random.normal(0, 1, n)
time_series = seasonal_pattern + noise

#time_series = np.ones(100) + 5

# Visualisieren der Zeitreihe
plt.plot(time_series)
plt.xlabel('Zeitschritte')
plt.ylabel('Wert')
plt.title('Zeitreihe mit saisonalem Muster')
plt.show()

# Anpassen eines ARIMA-Modells (p=12, d=0, q=0)
model = ARIMA(time_series, order=(16, 0, 0))
model_fit = model.fit()

# Vorhersagen für die nächsten k Schritte
k = 100
forecast = model_fit.forecast(steps=k)

# Visualisieren der Vorhersagen
next_steps = np.arange(n, n + k)
plt.plot(time_series, label='Beobachtungen')
plt.plot(next_steps, forecast, color='red', label='Vorhersagen')
plt.xlabel('Zeitschritte')
plt.ylabel('Wert')
plt.title('ARIMA-Vorhersage mit saisonalem Muster')
plt.legend()
plt.show()


<b>Effekt:</b> Die Autoregression berücksichtigt näher zurückliegende Datenpunkte mit höherem Gewicht. Es stellt sich im weiteren zeitlichen Prognoseraum der Durchschnitt vorhergehender Datenpunkte ein. Zyklische Aspekte werden hierbei nicht berücksichtigt.

### Vorhersagen aus Zeitriehen unter Berücksichtigung saisonaler Schankungen mit SARIMA-Modellen (Saisonal AutoRegressIve, Moving Average) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Erstellen einer Zeitreihe mit saisonalem Muster (z.B. jährlich)
np.random.seed(0)
n = 100
t = np.arange(n)
seasonal_pattern = 10 * np.sin(2 * np.pi * t / 12)  # Monatlich mit einer 12-monatigen Periode
noise = np.random.normal(0, 1, n)
time_series = seasonal_pattern + noise

# Visualisieren der Zeitreihe
plt.plot(time_series)
plt.xlabel('Zeitschritte')
plt.ylabel('Wert')
plt.title('Zeitreihe mit saisonalem Muster')
plt.show()

# Anpassen eines SARIMA-Modells mit saisonalen Parametern (p=1, d=0, q=0, P=1, D=1, Q=1, S=12)
model = SARIMAX(time_series, order=(1, 0, 0), seasonal_order=(1, 1, 1, 12))
model_fit = model.fit(disp=False)

# Vorhersagen für die nächsten k Schritte
k = 100
forecast = model_fit.get_forecast(steps=k)

# Visualisieren der Vorhersagen
next_steps = np.arange(n, n + k)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

plt.plot(time_series, label='Beobachtungen')
plt.plot(next_steps, forecast_mean, color='red', label='Vorhersagen')
#plt.fill_between(next_steps, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink', alpha=0.3, label='Konfidenzintervall')
plt.fill_between(next_steps, forecast_ci[:, 0], forecast_ci[:, 1], color='pink', alpha=0.3, label='Konfidenzintervall')

plt.xlabel('Zeitschritte')
plt.ylabel('Wert')
plt.title('SARIMA-Vorhersage mit saisonalem Muster')
plt.legend()
plt.show()



### SARIMA-Modelle funktionieren auch mit komplizierteren saisonalen Mustern... 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Erstellen einer Zeitreihe mit einem komplizierten saisonalen Muster
np.random.seed(0)
n = 300
t = np.arange(n)
seasonal_pattern = 20 * np.sin(2 * np.pi * t / 12) + (1*t/10) * np.cos(4 * np.pi * t / 12) + t/10 # Kombination aus Sinus- und Cosinus-Funktionen

#noise=0
noise = np.random.normal(0, 3, n)
time_series = seasonal_pattern + noise

# Visualisieren der Zeitreihe
plt.plot(time_series)
plt.xlabel('Zeitschritte')
plt.ylabel('Wert')
plt.title('Zeitreihe mit kompliziertem saisonalem Muster')
plt.show()

# Anpassen eines SARIMA-Modells mit saisonalen Parametern (p=2, d=0, q=2, P=1, D=1, Q=1, S=12)
#model = SARIMAX(time_series, order=(2, 0, 2), seasonal_order=(1, 1, 1, 12))

model = SARIMAX(time_series, order=(2, 0, 2), seasonal_order=(1, 1, 1, 12))

model_fit = model.fit(disp=False)

# Vorhersagen für die nächsten k Schritte
k = 100
forecast = model_fit.get_forecast(steps=k)

# Visualisieren der Vorhersagen
next_steps = np.arange(n, n + k)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

plt.plot(time_series, label='Beobachtungen')
plt.plot(next_steps, forecast_mean, color='red', label='Vorhersagen')
plt.fill_between(next_steps, forecast_ci[:, 0], forecast_ci[:, 1], color='pink', alpha=0.3, label='Konfidenzintervall')
plt.xlabel('Zeitschritte')
plt.ylabel('Wert')
plt.title('SARIMA-Vorhersage mit kompliziertem saisonalem Muster')
plt.legend()
plt.show()